In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import math
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from scipy import stats
import itertools as it
import sklearn.metrics as metrics
import sklearn.model_selection as selection
import os
import json
import pickle as pic
import importlib
import sys
sys.path.append("./../utils")
sys.path.append("./../sklearn")
import util
import constants as const
import ColumnTransformer
importlib.reload(util)
importlib.reload(const)
importlib.reload(ColumnTransformer)
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)
pd.set_option('precision', 6)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
#from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
import pickle
import helper

# 0. Some Constants and Setup

In [ ]:
paper_partitioners = [
    '1dd', 
    '1ds', 
    '2d', 
    '2ps', 
    'crvc', 
    'dbh', 
    'hdrf',
    'hep1',
    'hep10', 
    'hep100', 
    'ne',
] 

TARGET = "partitioning_time"
target = "partitioning_time"
num_parts = [4]

# 1. Load Data


In [ ]:
# Yes, we make copies. We will later filter. 
X_train = pd.read_csv("../datasets/graph-partitioning-run-time_train.csv")
y_train = X_train.copy()
# Yes, we make copies. We will later filter. 
X_test = pd.read_csv("../datasets/graph-partitioning-run-time_test.csv")
y_test = X_test.copy()

# 2. Helper
## 2.1 Helper for Evaluation

In [ ]:
def evaluate_model(model_name, model, X_test, y_test, target):
    """ We evalute the given model on the testset. 
    Args:
        model_name (string): the name of the model, e.g., XGB
        model (model): the trained model
        X_test (test data features): the testdata
        y_test (test data target): the testdata
        target (string): the name if the target column: "partitioning_time"
    Returns:
        list of dicts: the scores evaluated by partitioner and by graph 
    """
    # Which graphs are contained in the test set
    graphs = X_test.graph.unique()
    # Which partitioners are contained in the test set
    partitioners = X_test.partitioner.unique()

    results = []

    # 1. Get scores by partitioner and per test graph. This for loop also works if we only have on partitioner.
    for graph in graphs:
        for partitioner in partitioners:
            _X_test = X_test.copy()
            _y_test = y_test.copy()
            
            # Only evaluate the current graph on the current partitioner
            _X_test = _X_test[(_X_test.graph == graph)  & (_X_test.partitioner == partitioner)]
            _y_test = _y_test[(_y_test.graph == graph) & (_y_test.partitioner == partitioner)]

            # The predicted partitioning time
            _X_test["predicted_{}".format(target)] = model.predict(_X_test)

            # Set the true partitioning time
            _X_test[target] = _y_test[target]
            
            # Get the metrics/scores.
            metrics = helper.Metrics(target=target, predicted_target="predicted_{}".format(target))
            result = metrics.get_metrics(_X_test)

            result["partitioner"] = partitioner
            result["graph"] = graph.split("/")[-1]
            result["model"] = model_name
            
            results.append(result)

    # Since we also use this evaluation method for a model trained only for one single partitioner. 
    # In the next line we will evaluate how well the model performed for the different partitioner: One model multiple partitioners. 
    if (len(partitioners) > 1):
        # In this if we evaluate our model which we trained for different partitioners, not only for one partitioner.

        # 2. Get Score for each partitioner
        for partitioner in partitioners:
            _X_test = X_test.copy()
            _y_test = y_test.copy()
            
            # Only evaluate the current graph on the current partitioner
            _X_test = _X_test[(_X_test.partitioner == partitioner)]
            _y_test = _y_test[(_y_test.partitioner == partitioner)]

            # The predicted partitioning time
            _X_test["predicted_{}".format(target)] = model.predict(_X_test)

            # The true partitioning time
            _X_test[target] = _y_test[target]

            metrics = helper.Metrics(target=target, predicted_target="predicted_{}".format(target))
            result = metrics.get_metrics(_X_test)
            result["partitioner"] = partitioner
            result["graph"] = "all" # because the partitioner is evaluated on all graphs.
            result["model"] = model_name
            results.append(result)

        # 3. Get the overall score for all partitioners on all graphs for the given model
        _X_test = X_test.copy()
        _y_test = y_test.copy()

        # The predicted partitioning time
        _X_test["predicted_{}".format(target)] = model.predict(_X_test)
        _X_test[target] = _y_test[target]

        metrics = helper.Metrics(target, "predicted_{}".format(target))
        result = metrics.get_metrics(_X_test)
        result["partitioner"] = "all"
        result["graph"] = "all"
        result["model"] = model_name
        results.append(result)
    
    # IF We only train with one partitioner, then we can not set partitioner to "all". 
    else:
        # 3. Get for the one partitioner the scores. In this else we only have one partitioner which will be at index 0
        partitioner = partitioners[0]
        _X_test = X_test.copy()
        _y_test = y_test.copy()
    
        _X_test["predicted_{}".format(target)] = model.predict(_X_test)
        _X_test[target] = _y_test[target]

        metrics = helper.Metrics(target, "predicted_{}".format(target))
        result = metrics.get_metrics(_X_test)
        result["partitioner"] = partitioner
        result["graph"] = "all"
        result["model"] = model_name
        results.append(result)

    return results

## 2.2 Helper for Training 

In [ ]:
def train(model_name, model, X_train, y_train, X_test, y_test, target, store_model=False, result_file_name=""):
    """Traing the given pipeline on the training data and evaluate the model on the test set.

    Args:
        model_name (string): name of the model, e.g., rfr for random forest regressor
        model (pipeline): the pipeline to train. IT includes the grid search
        X_train (_type_): train data - features
        y_train (_type_): train data - target
        X_test (_type_): test data - features
        y_test (_type_): test data - target
        target (_type_): the target to predict: "partitioning_time"
        store_model (boolean): whether the best model in repsect to the achieved cross validation score should be saved
        result_file_name (string): where to store the model if it should be stored.

    Returns:
        list: the achieved scores
    """
    _X_train = X_train.copy()
    _y_train = y_train.copy()
    _X_test = X_test.copy()
    _y_test = y_test.copy()

    # Train the model incl. hyperparamter search. The hyperparameters are selected ONLY on the cross-validation score on the synthetic graphs.
    model.fit(_X_train, _y_train[target])

    print("PARAMETERS", model.get_params().keys())
    print("Best params", model.best_params_)
    print("Mean Test score", model.cv_results_["mean_test_score"])
    
    # Now we have the trained model
    # We store all the cross validation scores and also the best hyper-parameters
    # We further directly evaluate on the test set and also store these scores. 
    # IMPORTANT: Later on, we need to make the model selection / Hyperparamter search based on the crossvalidation scores, not on the test set of course. 

    # We serialize and store the best model
    if (store_model and not "" == result_file_name):
        pickle.dump(model, open(result_file_name, 'wb'))

    # Get the scores. 
    scores = evaluate_model(model_name, model, X_test=_X_test, y_test=_y_test, target=target)

    # Yes, for each score we add the cv score on the rmat graphs, the best hyperparameters. 
    # Because even if we evaluate on different graphs/partitioners, these score do not differ because the are based on synthetic graphs and lead to the model selection  
    for score in scores:
        score["rmat_best_params"] = model.best_params_
        score["rmat_cv_score"] =  model.cv_results_["mean_test_score"][model.best_index_]
        score["rmat_cv_scores"] =  model.cv_results_["mean_test_score"]
        score["rmat_best_index"] = model.best_index_
    return scores
   

# 3. Training

In [ ]:
graph_properties_simple = [
  'num_edges',
  'num_vertices',
]

graph_properties_easy = graph_properties_simple +  [
  "pearson_mode_degrees_in",
  "pearson_mode_degrees_out",
  "mean_degree",
  "density",
]  

graph_properties_hard = graph_properties_easy + [
    "mean_triangles", 
    "average_lcc",
]

featuresets = [
  graph_properties_simple, 
  graph_properties_easy, 
  graph_properties_hard, 
]


In [ ]:
overall_scores = []
PATH_STORE_MODEL = helper.create_results_folder("/home/ubuntu/cephstorage/partitioning-run-time-prediction-models")
PATH_STORE_SCORES = helper.create_results_folder("results-partitioning-run-time-prediction")

for USE_ALL_PARTITIONER in [
  True, 
  False
  ]:
  for model_name in [
    "KNN",
    "PolyRegression",
    "XGB", 
    "RFR", 
    "SVR"
    ]:
    print("We start the training with model ({}). We build ONE model for all partitioners: {}".format(model_name, USE_ALL_PARTITIONER))
    if USE_ALL_PARTITIONER:

      # We need to add the partitioner as a feature. Our trainings data already is one hot encoded but we need to allow the model to used the columns
      partitioner_one_hot_columns = paper_partitioners
      _featuresets = []
      for fs in featuresets:
        _featuresets.append(fs + partitioner_one_hot_columns)
      training_approach = "single-model-for-all-partitioners"

      # We use all the partitioners for the paper
      _X_train =  X_train[(X_train.partitioner.isin(paper_partitioners)) ]
      _y_train =  y_train[(y_train.partitioner.isin(paper_partitioners)) ]
      _X_test =  X_test[(X_test.partitioner.isin(paper_partitioners)) ]
      _y_test =  y_test[(y_test.partitioner.isin(paper_partitioners)) ]
      #print("_X_train with {} samples. {} are expected".format(len(_X_train), NUM_TRAIN_GRAPHS * NUM_PARTITIONERS))
      #print("_y_train with {} samples. {} are expected".format(len(_y_train), NUM_TRAIN_GRAPHS * NUM_PARTITIONERS))
      #print("_X_test with {} samples. {} are expected".format(len(_X_test), NUM_TEST_GRAPHS * NUM_PARTITIONERS))
      #print("_y_test with {} samples. {} are expected".format(len(_y_test), NUM_TEST_GRAPHS * NUM_PARTITIONERS))
      # We get the model which includes the gridsearch and pipeline
      model = helper.get_model(model_name, _featuresets)
      # Get scores: Will include that crossvalidation score for the best hyperparameters and also the scores for the test set
      # The testscores will be 1) per partitioner and per graph 2) per graph 3) per partitioner 4) overall score
      scores = train(
        model_name=model_name, 
        model=model, 
        X_train=_X_train, 
        y_train=_y_train, 
        X_test=_X_test, 
        y_test=_y_test, 
        target=TARGET, 
        store_model=True,
        result_file_name="{}/{}_{}".format(PATH_STORE_MODEL, training_approach, model_name))
      for score in scores:
        score["training_approach"] = training_approach
      overall_scores += scores
      pd.DataFrame(scores).to_csv("{}/{}_{}".format(PATH_STORE_SCORES, training_approach, model_name))
    
    else:
      training_approach = "single-model-per-partitioner"
      # For each partitioner we train a model 
      for partitioner in paper_partitioners:
          _X_train =  X_train[(X_train.partitioner == partitioner) ]
          _y_train =  y_train[(y_train.partitioner == partitioner)]
          _X_test =  X_test[(X_test.partitioner == partitioner)]
          _y_test =  y_test[(y_test.partitioner == partitioner)]
         # print("_X_train with {} samples. {} are expected".format(len(_X_train), NUM_TRAIN_GRAPHS))
         # print("_y_train with {} samples. {} are expected".format(len(_y_train),NUM_TRAIN_GRAPHS))
         # print("_X_test with {} samples. {} are expected".format(len(_X_test), NUM_TEST_GRAPHS))
         # print("_y_test with {} samples. {} are expected".format(len(_y_test), NUM_TEST_GRAPHS))
          # We get the model which includes the gridsearch and pipeline
          model = helper.get_model(model_name, featuresets)
          # Get scores: Will include that crossvalidation score for the best hyperparameters and also the scores for the test set. 
          scores = train(
            model_name=model_name,  
            model=model, 
            X_train=_X_train, 
            y_train=_y_train, 
            X_test=_X_test,
            y_test=_y_test, 
            target=TARGET,
            store_model=True,
            result_file_name="{}/{}_{}_{}".format(PATH_STORE_MODEL,training_approach, partitioner, model_name))
          for score in scores:
            score["training_approach"] = training_approach
          overall_scores += scores
          pd.DataFrame(scores).to_csv("{}/{}_{}_{}".format(PATH_STORE_SCORES, training_approach, partitioner, model_name))

scores_df = pd.DataFrame(overall_scores)
scores_df.to_csv("{}/scores-{}".format(PATH_STORE_SCORES, TARGET))

# 4. Evaluation
Have a look at the run-time-prediction-scores.ipynb with the scores reported in the paper

Use the correct time stamp

In [ ]:
scores_df = pd.read_csv("{}/scores-partitioning_time".format(PATH_STORE_SCORES))

Model selction based on rmat cross-validation scores

In [ ]:
# How well performed the model on average if evaluted for all graphs and all partitioners
scores_overall_df = scores_df[(scores_df.graph == "all") & (scores_df.partitioner == "all") & (scores_df.training_approach == "single-model-for-all-partitioners")]
NUMBER_MODELS = len(scores_overall_df.model.unique())
print("We trained {} models ({} expected) but select the best one based on crossvalidation on the rmat graphs".format(len(scores_overall_df), NUMBER_MODELS))
#Based on the cross valiudation score we select the best model. 
idx = scores_overall_df.groupby(["partitioner"])['rmat_cv_score'].transform(max) == scores_overall_df['rmat_cv_score']
# Now, we report the mape for the best model selected in the line above.
best_scores_overall_df = scores_overall_df[idx]
#print("MAPE: ", best_scores_overall_df["mape"].mean())
best_scores_overall_df[["r2", "rmse", "mape", "partitioner", "graph", "model", "rmat_cv_score", "training_approach", "rmat_best_params"]]